# AskNews

> [AskNews](https://asknews.app) infuses any LLM with the latest news, using a single natural language query. Specifically, AskNews is enriching over 300k articles per day, indexing them into a hot vector database, and putting that vector database on an endpoint for you. When you query AskNews, you get back a prompt-optimized string that goes directly into your prompt. This means that you do not need to manage your own news RAG, and you do not need to worry about how to properly convey news information in a condensed way to your LLM.

We can use this as a [retriever](/docs/modules/data_connection/retrievers). It will show functionality specific to this integration. After going through, it may be useful to explore [relevant use-case pages](/docs/use_cases/question_answering) to learn how to use this vectorstore as part of a larger chain.

## Setup

The integration lives in the `langchain-community` package. We also need to install the `asknews` package itself.

```bash
pip install -U langchain-community asknews
```

We also need to set our AskNews API credentials.

In [2]:
import getpass
import os

os.environ["ASKNEWS_CLIENT_ID"] = getpass.getpass()
os.environ["ASKNEWS_CLIENT_SECRET"] = getpass.getpass()

It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability

In [ ]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

## Usage

In [3]:
from langchain_community.retrievers import AskNewsRetriever

retriever = AskNewsRetriever(k=3)

retriever.invoke("influence of german politics on european finance")

[Document(page_content="[1]:\ntitle: Warning for Parties with Simple EU Solutions\nsummary: The EU has agreed to seize the Russian capital frozen in Western banks and use it to support Ukraine. However, this decision raises questions about the EU's commitment to upholding international law and agreements. The article argues that the EU's decision to take a simple solution to a complex problem may have unintended consequences. The author references the Brexit example, where the UK's decision to leave the EU led to significant economic and social consequences. The article concludes that simple solutions do not exist and that politicians should not be swayed by populist rhetoric.\nsource: Expressen\npublished: May 11 2024 04:15\nOrganization: EU\nNationality: Russian\nLocation: Ukraine, Western, UK\nLaw: international law\nPerson: author\nEvent: Brexit\nPolitics: politicians\nclassification: Politics\nsentiment: -1", metadata={'title': 'YLVA NILSSON: Varning för partier som har enkla EU-l

## Chaining

We can easily combine this retriever in to a chain.

In [5]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)
chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]) | retriever)
    | prompt
    | ChatOpenAI(model="gpt-4-1106-preview")
    | StrOutputParser()
)

In [8]:
chain.invoke({"question": "influence of german politics on european finance"})

"Based on the provided context, there are indications of the influence of German politics on European finance through the actions of Christian Lindner, the German Finance Minister. In the third document, it is mentioned that Lindner is pressuring two other German ministers to cut spending in the 2025 budget, highlighting the importance of supporting Ukraine and maintaining Germany's economic strength. Lindner's actions demonstrate Germany's fiscal prudence and its emphasis on economic stability, which can impact European finance considering Germany's significant role in the European Union's economy.\n\nFurthermore, Germany's stance on budgetary matters, especially in relation to supporting Ukraine amidst the ongoing tensions with Russia, is likely to influence the broader EU financial policies and priorities. However, the provided context does not detail specific mechanisms or decisions that directly link German politics to changes in European finance on a broader scale."